In [1]:
import scipy
import scipy.sparse
import cPickle
import logging
import implicit
reload(implicit)
import implicit.approximate_als
reload(implicit.approximate_als)
import implicit.bpr
reload(implicit.bpr)
import sklearn
import numpy as np
import itertools
import random
import pandas as pd
import os
import time
import lightfm
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import tqdm
import seaborn as sns
%matplotlib inline
import util as C
reload(C)
import functools
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import lightfm
import pymongo

# Load Data

In [2]:
logging.basicConfig(level = logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
channel_helper = C.ChannelHelper()
user_helper = C.UserHelper(channel_helper)

In [4]:
channel_helper.load_meta_data()
channel_helper.load_index_data()

In [5]:
user_helper.load_locale_data()

In [6]:
channel_cid2idx, channel_idx2cid = channel_helper.channel_cid2idx, channel_helper.channel_idx2cid
channel_data, channel_pid_data = channel_helper.channel_data, channel_helper.channel_pid_data

In [7]:
%%time
mat_csr = C.load_mat(C.LOCALE_ITEM_MAT_PATH)

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 21.5 ms


In [8]:
%%time
mat_coo = mat_csr.tocoo()

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 14.8 ms


In [9]:
%%time
mat_coo2 = implicit.nearest_neighbours.tfidf_weight(mat_coo)
mat_csr2 = mat_coo2.tocsr()

CPU times: user 536 ms, sys: 36 ms, total: 572 ms
Wall time: 568 ms


In [10]:
lfm_item_ids = np.arange(mat_csr2.shape[1])

# 按照国家和语言做排序

In [11]:
def list_top_channels(key, N = 10, verbose = True):
    key_idx = user_helper.get_locale_code(key)
    ws = -mat_csr[key_idx].toarray().ravel()
    idxs = ws.argpartition(N)[:N]
    scores = -ws[idxs]
    idxs = list(zip(idxs, scores))
    idxs.sort(key = lambda x: -x[1])
    res = []
    for idx, score in idxs:
        pid = channel_idx2cid[idx]
        title = channel_pid_data[pid]['title']
        res.append((pid, title, score))
    if verbose:
        for pid, title, value in res:
            print('pid={}, title={}, value={}'.format(pid, title, value))
    return res

In [12]:
_ = list_top_channels('zh/cn', N = 50)

pid=468563, title=Global News Podcast, value=2339.0
pid=570881, title=晓说2018, value=1990.0
pid=15554, title=段子来了, value=1838.0
pid=574393, title=6 Minute English, value=1743.0
pid=431972, title=TED Radio Hour, value=1513.0
pid=574416, title=Learning English for China, value=1486.0
pid=1086228, title=潘吉Jenny告诉你|学英语聊美国|开言英语 · Podcast, value=1264.0
pid=1024263, title=The Economist Radio (All audio), value=1208.0
pid=468582, title=Discovery, value=1199.0
pid=441712, title=果壳频道, value=1147.0
pid=1013607, title=英语口语每天学, value=1135.0
pid=20239, title=一席, value=1098.0
pid=935281, title=罗辑思维/得到App, value=1044.0
pid=1814, title=This American Life, value=1000.0
pid=431962, title=Planet Money, value=971.0
pid=574385, title=The English We Speak, value=963.0
pid=4541, title=TED Talks Daily, value=935.0
pid=1238672, title=60 Minutes, value=919.0
pid=14977, title=怀旧金曲, value=853.0
pid=20048, title=电影不无聊, value=834.0
pid=285553, title=罗辑思维, value=814.0
pid=943367, title=锵锵三人行, value=811.0
pid=18813, ti

# Evaluation Function

In [13]:
def compute_scores(self, user_id, user_items, recalculate_user=False):
    user = self._user_factor(user_id, user_items, recalculate_user = recalculate_user)
    scores = self.item_factors.dot(user)
    return scores

def eval_user_recomm(model, user_id, N = 10, verbose = True):
    if not isinstance(user_id, int):
        user_id = user_helper.get_locale_code(user_id)
    print('user_id = {}'.format(user_id))
    if model.__module__.startswith('implicit'):
        scores = -compute_scores(model, user_id, user_items = mat_csr2)
    else:
        scores = -model.predict(user_ids = user_id, item_ids = lfm_item_ids)
    pred_idxs = np.argpartition(scores, N)[:N]
    pred = [(x, scores[x]) for x in pred_idxs]
    pred.sort(key = lambda x: x[1])
    pred_idxs = [x[0] for x in pred]
    pred_pids = [channel_idx2cid[x] for x in pred_idxs]
    if verbose:
        for idx, pid in enumerate(pred_pids):
            print('- {} {} {} {}'.format(pid, channel_pid_data[pid]['title'], 
                                         pred_idxs[idx], pred[idx][1]))
    else:
        return pred_pids

# Train ALS Model

In [14]:
np.random.seed(42)
als_model = implicit.als.AlternatingLeastSquares(
    factors= 100, 
    iterations = 20)

In [15]:
%%time
als_model.fit(mat_csr2.T)

DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.062s
DEBUG:implicit:Calculated transpose in 0.019s
DEBUG:implicit:initialize factors in 0.264071941376
DEBUG:implicit:finished iteration 0 in 2.498s
DEBUG:implicit:finished iteration 1 in 2.502s
DEBUG:implicit:finished iteration 2 in 2.462s
DEBUG:implicit:finished iteration 3 in 2.507s
DEBUG:implicit:finished iteration 4 in 2.455s
DEBUG:implicit:finished iteration 5 in 2.386s
DEBUG:implicit:finished iteration 6 in 2.542s
DEBUG:implicit:finished iteration 7 in 2.509s
DEBUG:implicit:finished iteration 8 in 2.444s
DEBUG:implicit:finished iteration 9 in 2.560s
DEBUG:implicit:finished iteration 10 in 2.471s
DEBUG:implicit:finished iteration 11 in 2.570s
DEBUG:implicit:finished iteration 12 in 2.548s
DEBUG:implicit:finished iteration 13 in 2.414s
DEBUG:implicit:finished iteration 14 in 2.407s
DEBUG:implicit:finished iteration 15 in 2.536s
DEBUG:implicit:finished iteration 16 in 2.491s
DEBUG:implicit:fini

CPU times: user 5min 40s, sys: 7min 28s, total: 13min 8s
Wall time: 50.1 s


In [16]:
eval_user_recomm(als_model, 'zh/cn', N = 20)

user_id = 25
- 364268 The Psychology In Seattle Podcast 63070 -1.45106518269
- 115307 Doughboys 70491 -1.44882357121
- 535906 Change ma vie : Outils pour l'esprit 8789 -1.37248039246
- 381734 1001 Classic Short Stories & Tales 72794 -1.34524273872
- 32383 Géopolitique, le débat 22303 -1.32215833664
- 5995 Respectful Parenting: Janet Lansbury Unruffled 4581 -1.32071423531
- 8486 Show Time Spanish 6403 -1.31882667542
- 569596 The Vanished Podcast 43460 -1.31417274475
- 341806 Speak For Yourself with Cowherd & Whitlock 50599 -1.31044471264
- 130011 Forbes Audio Edition 78114 -1.30829644203
- 8469 Learn Portuguese - BrazilianPodClass 6389 -1.30688261986
- 109930 Kiss Kiss in the Mix cu Dj Yaang & OLiX 67578 -1.30198931694
- 32357 Monocle 24: The Globalist 22280 -1.29950714111
- 342014 La méthode scientifique 50683 -1.29528331757
- 2743 Collège de France (Histoire) 2106 -1.29195177555
- 40380 Liftoff 27477 -1.28773140907
- 130958 DJ AMICE 78594 -1.28479099274
- 382253 Official UEFA Champion

# Train BPR Model

In [19]:
np.random.seed(42)
bpr_model = implicit.bpr.BayesianPersonalizedRanking(
    factors= 100, 
    regularization = 0.01,
    iterations = 40)

In [20]:
%%time
bpr_model.fit(mat_csr2.T)

DEBUG:implicit:Converting input to COO format
DEBUG:implicit:Converted input to COO in 0.012s
DEBUG:implicit:fit epoch 0 in 0.168s (51.91% ranked correctly)
DEBUG:implicit:fit epoch 1 in 0.163s (54.24% ranked correctly)
DEBUG:implicit:fit epoch 2 in 0.165s (55.56% ranked correctly)
DEBUG:implicit:fit epoch 3 in 0.164s (56.63% ranked correctly)
DEBUG:implicit:fit epoch 4 in 0.165s (57.43% ranked correctly)
DEBUG:implicit:fit epoch 5 in 0.162s (58.24% ranked correctly)
DEBUG:implicit:fit epoch 6 in 0.164s (58.88% ranked correctly)
DEBUG:implicit:fit epoch 7 in 0.168s (59.60% ranked correctly)
DEBUG:implicit:fit epoch 8 in 0.164s (60.36% ranked correctly)
DEBUG:implicit:fit epoch 9 in 0.165s (61.23% ranked correctly)
DEBUG:implicit:fit epoch 10 in 0.163s (61.99% ranked correctly)
DEBUG:implicit:fit epoch 11 in 0.164s (62.75% ranked correctly)
DEBUG:implicit:fit epoch 12 in 0.165s (63.56% ranked correctly)
DEBUG:implicit:fit epoch 13 in 0.164s (64.19% ranked correctly)
DEBUG:implicit:fit e

CPU times: user 1min 45s, sys: 88 ms, total: 1min 46s
Wall time: 7.27 s


In [22]:
eval_user_recomm(bpr_model, 'zh/cn', N = 20)

user_id = 29
- 16321 投資新世代 11985 -6.23288106918
- 15654 轻松一刻粤语版 11592 -6.22307109833
- 320815 Bilgi Çağı 39628 -6.20479774475
- 19708 汽车入门学校 14142 -6.16602802277
- 1077062 放映邪会 - 动漫电台 20043 -6.1393866539
- 1199276 久米宏　ラジオなんですけど 89716 -6.12867832184
- 15030 你的故事我的歌(新) 11186 -6.12150764465
- 4352 Tokyo Midtown presents The Lifestyle MUSEUM 3314 -6.11914205551
- 973833 能猫老徐 119527 -6.07099723816
- 14010 《 自然療法與你 》 - 源網台 sourcewadio.com 10569 -6.04407787323
- 14409 90日聆聽新約聖經 10819 -6.03264093399
- 1017705 深海之蓝_Ryan 99348 -6.02314853668
- 13285 催眠放松 10063 -6.00807571411
- 14992 音乐大明星 11158 -6.00692224503
- 485611 《赖世雄美语从头学初级美语（上）》讲解音频 123842 -5.99964284897
- 14966 ちょっとエッチなクラブ学 横山美雪のこんなの初めてっ!![FLOORnet podcast] 11137 -5.99943161011
- 16881 极客方式 12367 -5.99635505676
- 13845 ❤恋愛心理学❤知りたい！相手の気持ち 10461 -5.99033069611
- 366629 鞠强教授从管理心理学视角谈历史 6796 -5.98399162292
- 1202824 为你读英语美文 91398 -5.98353481293


# LightFM Model

In [23]:
%%time
lfm_model = lightfm.LightFM(no_components=100, loss = 'warp', random_state=42)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 67 µs


In [24]:
%%time
lfm_model.fit_partial(mat_csr2, epochs = 20, num_threads = 8, verbose = True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
CPU times: user 4min 2s, sys: 208 ms, total: 4min 3s
Wall time: 33.5 s


In [28]:
%%time
eval_user_recomm(lfm_model, 'en/us', N = 50)

user_id = 0
- 1294718 11 Inicial 134174 -1.2246556282
- 376472 The War Room 71327 -1.05111908913
- 16920 Emacs Chat 12393 0.182210057974
- 1160312 How Is She Though?: RuPaul's Drag Race Recap 133008 0.518623411655
- 535552 Roaring Riot Podcast 58863 0.52796214819
- 961233 Erotica Magazine: Playboy Lifestyle | Erotic Audio |Erotic Models | How to Pick Up Girls » Toy (Erotica Magazine Audio) 21825 0.588945269585
- 1174524 Under the Hood Podcast 77620 0.635321080685
- 206955 Plural of You 110128 0.644687891006
- 1246086 El Petroleo es Nuestro: A History of Oil in Mexico 112161 0.671018302441
- 522362 Conversations With Writers 139072 0.704585134983
- 452525 Celtics Life NBA Basketball Podcast 108857 0.725243270397
- 1290875 UFC Reviews and After Show 132915 0.729335010052
- 435698 PeaceCast 100782 0.743344306946
- 420945 The Home and the World by Rabindranath Tagore 97531 0.753208637238
- 7412 Heavens Above Astronomy Podcast 5677 0.784580945969
- 193363 Helluva Catholic Podcast 110209 0.7

# 横向对比

In [29]:
user_helper.locales

Index([u'en/us', u'en/gb', u'en/au', u'en/ca', u'ko/kr', u'sv/se', u'pt/br',
       u'ru/ru', u'de/de', u'ja/jp',
       ...
       u'en/ly', u'fr/ga', u'ar/de', u'en/al', u'es/br', u'en/bb', u'ar/lb',
       u'it/ch', u'en/tn', u'fr/mu'],
      dtype='object', length=257)

In [31]:
locale = 'zh/cn'
n = 40
print('=' * 10 + '\nlist top channels')
_ = list_top_channels(locale, N = n)
print('=' * 10 + '\nlfm model')
eval_user_recomm(lfm_model, locale, N = n)

list top channels
pid=468563, title=Global News Podcast, value=3230.0
pid=15554, title=段子来了, value=3214.0
pid=570881, title=晓说2018, value=2644.0
pid=13818, title=机核网 GADIO 游戏广播, value=2565.0
pid=431972, title=TED Radio Hour, value=2269.0
pid=574393, title=6 Minute English, value=2105.0
pid=574416, title=Learning English for China, value=1846.0
pid=1024263, title=The Economist Radio (All audio), value=1777.0
pid=20239, title=一席, value=1725.0
pid=20048, title=电影不无聊, value=1703.0
pid=4541, title=TED Talks Daily, value=1609.0
pid=1086228, title=潘吉Jenny告诉你|学英语聊美国|开言英语 · Podcast, value=1557.0
pid=468582, title=Discovery, value=1447.0
pid=943367, title=锵锵三人行, value=1373.0
pid=441712, title=果壳频道, value=1350.0
pid=1013607, title=英语口语每天学, value=1342.0
pid=4222, title=友的聊播客, value=1282.0
pid=935281, title=罗辑思维/得到App, value=1271.0
pid=1814, title=This American Life, value=1253.0
pid=1216928, title=Stuff You Should Know, value=1250.0
pid=574385, title=The English We Speak, value=1245.0
pid=131574, 

# 部署到数据库

In [17]:
def pid_to_key(pid):
    assert(pid in channel_pid_data)
    data = channel_pid_data[pid]
    return data['key']

def get_track_keys(feeds):
    tracks = []
    for x in feeds:
        track = channel_data.get(x, {}).get('track')
        if not track: continue
        tracks.append('{}-{}'.format(x, track))
    return tracks

def deploy_to_db(table, recomm_data):
    ops = []
    for (f, feeds, tracks) in tqdm.tqdm(recomm_data):
        ops.append(pymongo.UpdateOne({'_id': f}, {'$set': {'feeds': feeds, 'tracks': tracks}}, upsert=True))
        if len(ops) == 100:
            table.bulk_write(ops, ordered=False)
            ops = []
    if ops:
        table.bulk_write(ops, ordered=False)

In [18]:
recomm_data = []
for locale in tqdm.tqdm(user_helper.locales):
    res = list_top_channels(locale, N = 200, verbose = False)
    pids = [x[0] for x in res]
    keys = [pid_to_key(x) for x in pids]
    tracks = get_track_keys(keys)
    recomm_data.append((locale, keys, tracks))

100%|██████████| 311/311 [00:00<00:00, 526.42it/s]


In [19]:
from recomm import app
table = app.CBRecommDB['user_locale']
# table.create_index('_id')
deploy_to_db(table, recomm_data)

/home/ubuntu/nova/venv/local/lib/python2.7/site-packages/flask_cache/jinja2ext.py:33: ExtDeprecationWarning: Importing flask.ext.cache is deprecated, use flask_cache instead.
  from flask.ext.cache import make_template_fragment_key
100%|██████████| 311/311 [00:00<00:00, 1058.55it/s]
